<a href="https://colab.research.google.com/github/AnyiChomtin/FlightDelayProphet/blob/main/Flight_Delay_Prophet_EDA_%26_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project Motivation

As a student living in the San Francisco Bay Area, traveling by air is a popular choice. However, there is one thing annoying about it, which is flight delays. We don't know whether there will be a delay, how long the delay might be, and we have no idea when the notification of delay will be sent out, and it can be a nightmare when we are in a cab to the airport and receive a text message saying that "your flight XXXXXX has delayed 3 hours, and we apologize for any inconvenience". Yes, that can be really incovenient if you have to spent 3 more hours waiting in the lounge. No seriously, that can actually feel like 8 hours.

Therefore, our group is interested in building a flight delay prophet, which can help travellers in the bay area to predict delays even before they leave home, so that they might be able to have a better understanding of the possibility of a delay for their flight and how bad that might be, and therefore have a wiser plan and use of their time instead of getting stuck in the excess waiting in the airport. We believe this project might be valuable since it can save travellers much time, and that can be considerably impactful if those saved time accumulated in each day. If our model is proven to provide some great insights, it might also be inspiring for other areas or even globally to research related features. Moreover, it can also be connected with models on other related areas, such as those predicting commuting time to the airport, which might provide travellers with more precise time management.

Let's get started.

### Initialization

Setting up our working directory and importing the libraries we need for EDA and Data Pre-processing:

In [ ]:
import os
os.getcwd()

'/content/drive/My Drive/ieor 142 final project/Flight Delay/Original Data'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
new_path = '/content/drive/My Drive/ieor 142 final project/Flight Delay/Original Data'
os.chdir(new_path)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Importing Datasets

To start with, we found two datasets that might be valuable.

The first one is a datasets about all U.S. domestic flight delays from January 2019 to June 2019: https://www.kaggle.com/datasets/undersc0re/flight-delay-and-causes

 It contains different types of information of the flight, from the time, airplane, and distance, etc. We believe that from transforming and exploring these features, we can get some information that might be useful for predicting flight delays.

Later, we will add other dataset to make our model more robust.

In [ ]:
flight_delay = pd.read_csv("Flight_delay.csv")
print(flight_delay.describe())
flight_delay.info()
print(flight_delay.shape)
flight_delay.head()

           DayOfWeek        DepTime        ArrTime     CRSArrTime  \
count  484551.000000  484551.000000  484551.000000  484551.000000   
mean        3.991605    1564.477865    1617.784438    1652.129929   
std         1.971466     452.235219     583.637660     466.096216   
min         1.000000       1.000000       1.000000       1.000000   
25%         2.000000    1234.000000    1327.000000    1339.000000   
50%         4.000000    1620.000000    1737.000000    1723.000000   
75%         6.000000    1928.000000    2049.000000    2025.000000   
max         7.000000    2400.000000    2400.000000    2359.000000   

           FlightNum  ActualElapsedTime  CRSElapsedTime        AirTime  \
count  484551.000000      484551.000000   484551.000000  484551.000000   
mean     2139.207386         134.810422      131.400761     108.877134   
std      1812.677071          74.070374       71.542531      70.113513   
min         1.000000          15.000000      -21.000000       0.000000   
25%     

,DayOfWeek,Date,DepTime,ArrTime,CRSArrTime,UniqueCarrier,Airline,FlightNum,TailNum,ActualElapsedTime,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,4,03-01-2019,1829,1959,1925,WN,Southwest Airlines Co.,3920,N464WN,90,...,3,10,0,N,0,2,0,0,0,32
1,4,03-01-2019,1937,2037,1940,WN,Southwest Airlines Co.,509,N763SW,240,...,3,7,0,N,0,10,0,0,0,47
2,4,03-01-2019,1644,1845,1725,WN,Southwest Airlines Co.,1333,N334SW,121,...,6,8,0,N,0,8,0,0,0,72
3,4,03-01-2019,1452,1640,1625,WN,Southwest Airlines Co.,675,N286WN,228,...,7,8,0,N,0,3,0,0,0,12
4,4,03-01-2019,1323,1526,1510,WN,Southwest Airlines Co.,4,N674AA,123,...,4,9,0,N,0,0,0,0,0,16


The second dataset is a weather indicator datasets for each airport daily. We downloaded this data from National Centers for Environmental Information: https://www.ncdc.noaa.gov/cdo-web/search

We selected the 3 ZIP codes (94128, 94621, 95110) that the three bay area airports are in for the entire Year 2019. With more investigation, we noticed that these ZIP codes covers the airport areas only so we can say they are specifically for the airports.

In [ ]:
bay_weather = pd.read_csv("Bay Area Airport Weather Data 2019.csv")
print(bay_weather.describe())
bay_weather.info()
print(bay_weather.shape)
bay_weather.head()

              AWND         PGTM         PRCP   SNOW   SNWD        TAVG  \
count  1094.000000    98.000000  1095.000000  307.0  359.0  365.000000   
mean      8.671700  1476.857143     0.054420    0.0    0.0   58.654795   
std       3.985422   355.556106     0.171066    0.0    0.0    6.334647   
min       1.340000     2.000000     0.000000    0.0    0.0   45.000000   
25%       5.820000  1359.000000     0.000000    0.0    0.0   55.000000   
50%       8.050000  1539.000000     0.000000    0.0    0.0   58.000000   
75%      10.740000  1653.000000     0.000000    0.0    0.0   63.000000   
max      25.050000  2332.000000     1.640000    0.0    0.0   79.000000   

              TMAX         TMIN         WDF2         WDF5         WSF2  \
count  1092.000000  1094.000000  1095.000000  1088.000000  1095.000000   
mean     69.158425    51.471664   263.954338   263.152574    20.064110   
std       9.858652     6.780753    72.971519    73.104924     6.240078   
min      50.000000    33.000000    10

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT05,WT08
0,USW00023293,"SAN JOSE, CA US",2019-01-01,5.82,NaN,0.0,NaN,NaN,NaN,55.0,36.0,50,60.0,17.0,23.0,NaN,NaN,NaN,NaN,NaN
1,USW00023293,"SAN JOSE, CA US",2019-01-02,4.03,NaN,0.0,NaN,NaN,NaN,57.0,33.0,330,330.0,8.9,10.1,NaN,NaN,NaN,NaN,NaN
2,USW00023293,"SAN JOSE, CA US",2019-01-03,2.68,NaN,0.0,NaN,NaN,NaN,58.0,34.0,320,340.0,8.9,11.0,NaN,NaN,NaN,NaN,NaN
3,USW00023293,"SAN JOSE, CA US",2019-01-04,3.36,NaN,0.0,NaN,NaN,NaN,62.0,35.0,200,200.0,14.1,19.0,NaN,NaN,NaN,NaN,NaN
4,USW00023293,"SAN JOSE, CA US",2019-01-05,13.20,NaN,0.1,NaN,NaN,NaN,56.0,46.0,130,150.0,30.0,40.0,1.0,NaN,NaN,NaN,1.0


### Data Pre-processing

#### Flight Delay Data - Data Cleaning

*This* section focuses on identifying flights originating from or destined for airports in the San Francisco Bay Area.

In [ ]:
# This line creates a list named 'bay_area_airports' containing the three-letter codes for airports in the San Francisco Bay Area
bay_area_airports = ['SFO', 'OAK', 'SJC']

# This line filters the 'flight_delay' DataFrame to include only flights either originating from or destined to a Bay Area airport
#       - Flights where the 'Origin' column value is in the 'bay_area_airports' list
#       - Flights where the 'Dest' column value is in the 'bay_area_airports' list
bay_area_flights = flight_delay[(flight_delay['Origin'].isin(bay_area_airports)) |
                                (flight_delay['Dest'].isin(bay_area_airports))]

print(bay_area_flights.shape)
print(bay_area_flights.head())

(50660, 29)
    DayOfWeek        Date  DepTime  ArrTime  CRSArrTime UniqueCarrier  \
77          4  03-01-2019     1944     2110        1915            WN   
78          4  03-01-2019     1251     1425        1345            WN   
79          4  03-01-2019     1548     1728        1635            WN   
80          4  03-01-2019     2216     2348        2255            WN   
81          4  03-01-2019     2010     2203        2030            WN   

                   Airline  FlightNum TailNum  ActualElapsedTime  ...  TaxiIn  \
77  Southwest Airlines Co.        223  N223WN                 86  ...       5   
78  Southwest Airlines Co.        237  N281WN                 94  ...       3   
79  Southwest Airlines Co.        280  N460WN                100  ...       4   
80  Southwest Airlines Co.        900  N679AA                 92  ...       4   
81  Southwest Airlines Co.        962  N378SW                113  ...      28   

    TaxiOut  Cancelled  CancellationCode Diverted CarrierDelay

This section focuses on cleaning and organizing the `bay_area_flights` DataFrame, specifically the `'Date'` column.

1. **Date Format Validation:**


```
bay_area_flights[bay_area_flights['Date'].str.match(r'^\d{2}-\d{2}-\d{4}$')]
```
- This line checks the format of the `'Date'` column in the `bay_area_flights` DataFrame.
- It utilizes the `str.match` function with a regular expression (`r'^\d{2}-\d{2}-\d{4}$'`) to ensure the dates follow a specific format:
  - `^`: Matches the beginning of the string.
  - `\d{2}`: Matches two digits, representing the day of the month.
  - `-`: Matches a hyphen separator.
  - Repeated twice more for month and year, ensuring a consistent DD-MM-YYYY format.

2. **Converting Dates to DateTime:**


```
pd.to_datetime(bay_area_flights['Date'], format='%d-%m-%Y')
```
- Assuming the date format validation is successful, we proceed to convert the `'Date'` column from strings to datetime objects.
- The `pd.to_datetime` function is used for this purpose.
- This conversion allows for more advanced date and time manipulation and analysis later in the process.

3. **Sorting by Date:**


```
bay_area_flights.sort_values(by='Date')
```
- Finally, we sort the `bay_area_flights` DataFrame by the `'Date'` column in ascending order.
- Sorting by date ensures a chronological order, which is beneficial for further analysis or visualization tasks.


In [ ]:
bay_area_flights = bay_area_flights[bay_area_flights['Date'].str.match(r'^\d{2}-\d{2}-\d{4}$')]
bay_area_flights['Date'] = pd.to_datetime(bay_area_flights['Date'], format='%d-%m-%Y')
bay_area_flights = bay_area_flights.sort_values(by='Date')
print(bay_area_flights.shape)
bay_area_flights.tail()

(50660, 29)


,DayOfWeek,Date,DepTime,ArrTime,CRSArrTime,UniqueCarrier,Airline,FlightNum,TailNum,ActualElapsedTime,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
434819,1,2019-06-30,1155,1243,1157,OO,Skywest Airlines Inc.,5737,N292SW,48,...,2,12,0,N,0,0,0,0,0,46
434818,1,2019-06-30,1257,1351,1317,OO,Skywest Airlines Inc.,5737,N292SW,54,...,4,6,0,N,0,0,0,0,0,34
434817,1,2019-06-30,1722,1856,1836,OO,Skywest Airlines Inc.,5728,N218SW,94,...,4,17,0,N,0,19,0,1,0,0
434827,1,2019-06-30,1440,1547,1407,OO,Skywest Airlines Inc.,5756,N299SW,67,...,2,32,0,N,0,0,0,11,0,89
434752,1,2019-06-30,1004,1123,1022,OO,Skywest Airlines Inc.,5474,N301YV,79,...,6,20,0,N,0,61,0,0,0,0


In [ ]:
bay_area_flights['DepTime'].sort_values().head()

118918    1
394856    1
119795    1
209387    1
19279     1
Name: DepTime, dtype: int64

## Cleaning Departure and Arrival Times

This section addresses the `'DepTime'` (departure time) and `'ArrTime'` (arrival time) columns in the `bay_area_flights` DataFrame.
At the end we'll ensure a clean and standardized format for departure and arrival times, facilitating further analysis.

**Workflow:**

1. **Leading Zero Padding:**
   - Ensures a consistent four-digit format (e.g., "0915") for all times using `f-strings`.

2. **Time Validation:**
   - Checks if hours (`< 24`) and minutes (`< 60`) fall within valid ranges.
   - Invalid entries are replaced with `"0000"` for identification.

3. **Conversion to Time Objects:**
   - Converts time strings to datetime time objects using `pd.to_datetime` with format `'%H%M'`.
   - Handles potential errors (`'coerce'`) by converting them to `NaT` (Not a Time).
   - Extracts the time portion using `.dt.time` for dedicated time data.


In [ ]:
bay_area_flights['DepTime'] = bay_area_flights['DepTime'].apply(lambda x: f"{int(x):04d}")
bay_area_flights['DepTime'] = bay_area_flights['DepTime'].apply(lambda x: x if int(x[:2]) < 24 and int(x[2:]) < 60 else '0000')
bay_area_flights['DepTime'] = pd.to_datetime(bay_area_flights['DepTime'], format='%H%M', errors='coerce').dt.time

bay_area_flights['ArrTime'] = bay_area_flights['ArrTime'].apply(lambda x: f"{int(x):04d}")
bay_area_flights['ArrTime'] = bay_area_flights['ArrTime'].apply(lambda x: x if int(x[:2]) < 24 and int(x[2:]) < 60 else '0000')
bay_area_flights['ArrTime'] = pd.to_datetime(bay_area_flights['ArrTime'], format='%H%M', errors='coerce').dt.time

print(bay_area_flights[['Date', 'Airline', 'DepTime', 'ArrTime', 'ArrDelay', 'DepDelay']].head())

            Date                 Airline   DepTime   ArrTime  ArrDelay  \
83187 2019-01-01         JetBlue Airways  18:56:00  22:33:00        23   
38707 2019-01-01   United Air Lines Inc.  23:47:00  01:11:00       237   
45314 2019-01-01  Southwest Airlines Co.  19:02:00  21:06:00        31   
45377 2019-01-01  Southwest Airlines Co.  15:48:00  17:10:00        35   
45388 2019-01-01  Southwest Airlines Co.  21:52:00  23:15:00        35   

       DepDelay  
83187        31  
38707       253  
45314        42  
45377        43  
45388        47  


In [ ]:
bay_area_flights_w_new_features = bay_area_flights.copy()

Now, let's start adding some features of our interest based on the columns we have. First, let's extract the hour in the day that each flight is at:

In [ ]:
def extract_hour(dep_time):
    return dep_time.hour
bay_area_flights_w_new_features['DepHour'] = bay_area_flights_w_new_features['DepTime'].apply(extract_hour)
print(bay_area_flights_w_new_features.shape)
bay_area_flights_w_new_features[['Date', 'Airline', 'DepTime', 'DepHour', 'ArrDelay', 'DepDelay']].head()

(50660, 30)


,Date,Airline,DepTime,DepHour,ArrDelay,DepDelay
83187,2019-01-01,JetBlue Airways,18:56:00,18,23,31
38707,2019-01-01,United Air Lines Inc.,23:47:00,23,237,253
45314,2019-01-01,Southwest Airlines Co.,19:02:00,19,31,42
45377,2019-01-01,Southwest Airlines Co.,15:48:00,15,35,43
45388,2019-01-01,Southwest Airlines Co.,21:52:00,21,35,47


Based on our observation, there seems to be a lot of 0 values for the cancellation and diverted columns, let's check it and remove them if that is the case:

In [ ]:
print(bay_area_flights_w_new_features['Cancelled'].mean())
print(bay_area_flights_w_new_features['CancellationCode'].unique())
print(bay_area_flights_w_new_features['Diverted'].mean())

0.0
['N']
0.0


In [ ]:
bay_area_flights_w_new_features = bay_area_flights_w_new_features.drop(columns=['Cancelled', 'CancellationCode', 'Diverted'])

Transform the day of week column to make it more readable:

In [ ]:
day_of_week_mapping = {
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday',
    7: 'Sunday'
}

bay_area_flights_w_new_features['DayOfWeek'] = bay_area_flights_w_new_features['DayOfWeek'].map(day_of_week_mapping)
bay_area_flights_w_new_features['Month'] = bay_area_flights_w_new_features['Date'].dt.month
print(bay_area_flights_w_new_features[['Date', 'Month', 'DayOfWeek', 'Airline', 'DepTime', 'ArrDelay', 'DepDelay']].head())

            Date  Month DayOfWeek                 Airline   DepTime  ArrDelay  \
83187 2019-01-01      1   Tuesday         JetBlue Airways  18:56:00        23   
38707 2019-01-01      1   Tuesday   United Air Lines Inc.  23:47:00       237   
45314 2019-01-01      1   Tuesday  Southwest Airlines Co.  19:02:00        31   
45377 2019-01-01      1   Tuesday  Southwest Airlines Co.  15:48:00        35   
45388 2019-01-01      1   Tuesday  Southwest Airlines Co.  21:52:00        35   

       DepDelay  
83187        31  
38707       253  
45314        42  
45377        43  
45388        47  


## Exploring Potential Delay Dependencies

While we've cleaned and organized the flight data, further analysis can be conducted to identify potential factors influencing delays. One possibility is the delay of a plane impacting its subsequent flights on the same day.

To investigate this, we can sort the `bay_area_flights` DataFrame by the following criteria:

1. **Date:** This ensures a chronological order, allowing us to track a specific plane's flights throughout the day.
2. **Origin:** Grouping by origin helps identify flights originating from the same airport, potentially involving the same plane.
3. **Departure Time:** Sorting by departure time within each origin group facilitates tracing the sequence of a plane's flights.

By implementing this multi-level sorting, we can analyze potential correlations between a flight's delay and the delay of the same plane's earlier flight on the same day.

In [ ]:
bay_area_flights_w_new_features.sort_values(by=['Date', 'Origin', 'DepTime'], inplace=True)
print(bay_area_flights_w_new_features.shape)
print(bay_area_flights_w_new_features[['Date', 'Origin', 'DepTime', 'ArrDelay', 'DepDelay']].head())

(50660, 28)
            Date Origin   DepTime  ArrDelay  DepDelay
85842 2019-01-01    ATL  14:52:00        23         7
85917 2019-01-01    ATL  19:31:00        59        51
72263 2019-01-01    AUS  16:23:00        62        78
17326 2019-01-01    BOI  14:47:00        54        64
17313 2019-01-01    BOI  15:30:00        15         7


In [ ]:
bay_area_flights_w_new_features.head()

,DayOfWeek,Date,DepTime,ArrTime,CRSArrTime,UniqueCarrier,Airline,FlightNum,TailNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepHour,Month
85842,Tuesday,2019-01-01,14:52:00,17:21:00,1658,DL,Delta Air Lines Inc.,629,N648DL,329,...,2139,6,20,7,0,16,0,0,14,1
85917,Tuesday,2019-01-01,19:31:00,22:03:00,2104,DL,Delta Air Lines Inc.,1155,N3762Y,332,...,2116,6,17,0,0,8,0,51,19,1
72263,Tuesday,2019-01-01,16:23:00,17:52:00,1650,AA,American Airlines Inc.,1023,N589AA,209,...,1476,2,12,25,0,0,0,37,16,1
17326,Tuesday,2019-01-01,14:47:00,15:24:00,1430,OO,Skywest Airlines Inc.,6383,N732SK,97,...,522,4,8,0,0,0,0,54,14,1
17313,Tuesday,2019-01-01,15:30:00,16:26:00,1611,OO,Skywest Airlines Inc.,6338,N923SW,116,...,522,7,19,0,0,0,0,15,15,1


## Creating Feature: Same Day Flights per Plane

This section introduces a new feature, `"Same_Day_Num_Flights_of_this_Plane"`, to the `bay_area_flights` DataFrame. This feature aims to capture the number of flights a particular plane has on the same day.

**Workflow:**

1. **Group by Date and TailNum:**
   - We can utilize the `groupby` function on the `'Date'` and `'TailNum'` columns.
   - This groups the DataFrame by flights occurring on the same date and involving the same plane (identified by the `"TailNum"`).

2. **Count Flights per Group:**
   - Within each group, we can calculate the number of flights for that specific plane on that day.
   - The `size` attribute applied to the group object provides this count.

3. **Adding the Feature:**
   - After iterating through the groups and calculating flight counts, we can assign a new column named `"Same_Day_Num_Flights_of_this_Plane"` to the DataFrame.
   - This column will contain the count values for each flight, indicating the number of flights for that plane on the same day.

In [ ]:
bay_area_flights_w_new_features['Date'] = pd.to_datetime(bay_area_flights_w_new_features['Date'])
bay_area_flights_w_new_features['Same_Day_Num_Flights_of_this_Plane'] = bay_area_flights_w_new_features.groupby(['Date', 'TailNum'])['FlightNum'].transform('count')
print(bay_area_flights_w_new_features.head())

      DayOfWeek       Date   DepTime   ArrTime  CRSArrTime UniqueCarrier  \
85842   Tuesday 2019-01-01  14:52:00  17:21:00        1658            DL   
85917   Tuesday 2019-01-01  19:31:00  22:03:00        2104            DL   
72263   Tuesday 2019-01-01  16:23:00  17:52:00        1650            AA   
17326   Tuesday 2019-01-01  14:47:00  15:24:00        1430            OO   
17313   Tuesday 2019-01-01  15:30:00  16:26:00        1611            OO   

                      Airline  FlightNum TailNum  ActualElapsedTime  ...  \
85842    Delta Air Lines Inc.        629  N648DL                329  ...   
85917    Delta Air Lines Inc.       1155  N3762Y                332  ...   
72263  American Airlines Inc.       1023  N589AA                209  ...   
17326   Skywest Airlines Inc.       6383  N732SK                 97  ...   
17313   Skywest Airlines Inc.       6338  N923SW                116  ...   

       TaxiIn  TaxiOut  CarrierDelay  WeatherDelay NASDelay SecurityDelay  \
85842    

In [ ]:
bay_area_flights_w_new_features.to_csv('/content/drive/My Drive/ieor 142 final project/Flight Delay/Transformed Data/Bay_Area_Flights.csv', index=False)

#### Flight Delay Data - Adding Features

## Feature Engineering: Previous Flight Delay

This section introduces a new feature, `"Previous_Flight_Delay"`, to the `bay_area_flights` DataFrame. This feature aims to capture the delay of the most recent flight the same plane had on the same day, potentially influencing the current flight's delay.

**Workflow:**

1. **Sorting by Date, Origin, and Departure Time:**
   - As discussed earlier, we sort the DataFrame by `'Date'`, `'Origin'`, and `'DepTime'`.
   - This ensures a chronological order within each plane's flights on a specific day.

2. **Iterating with `shift`:**
   - We can leverage the `shift` function while iterating through the DataFrame.
   - The `shift` function allows us to access values in previous rows based on a specified number of positions.

3. **Identifying Previous Flight Delay:**
   - Within the loop, we can use `df['TailNum'].shift(1)` to access the `"TailNum"` of the previous flight in the sorted order.
   - We then compare this shifted `"TailNum"` with the current row's `"TailNum"`.
   - If they match, it indicates the previous flight for the same plane on the same day.
   - In this case, we can access the corresponding delay value from the shifted row using `df['Delay'].shift(1)`.

4. **Handling First Flight:**
   - If the shifted `"TailNum"` doesn't match, it signifies the current flight is the first for the plane that day.
   - In this scenario, we assign a delay value of 0 (no previous flight on the same day).

5. **Assigning New Feature:**
   - By iterating through the DataFrame, we can create a new column named `"Previous_Flight_Delay"`.
   - This column will hold the delay of the most recent previous flight for each plane on the same day, or 0 for first flights.

By incorporating this feature, we can analyze if the delay of a plane's earlier flight on the same day might influence the delay of its current flight. This exploration helps build a more comprehensive understanding of potential factors impacting flight delays.


In [ ]:
bay_area_flights_w_new_features['ArrDelay'] = pd.to_numeric(bay_area_flights_w_new_features['ArrDelay'], errors='coerce')
bay_area_flights_w_new_features['DepDelay'] = pd.to_numeric(bay_area_flights_w_new_features['DepDelay'], errors='coerce')
bay_area_flights_w_new_features['TotalDelay'] = bay_area_flights_w_new_features['ArrDelay'] + bay_area_flights_w_new_features['DepDelay']

In [ ]:
bay_area_flights_w_new_features.sort_values(by=['Date', 'TailNum', 'DepTime'], inplace=True)
shifted_delays = bay_area_flights_w_new_features.groupby(['Date', 'TailNum']).shift(-1)
bay_area_flights_w_new_features['Plane_Same_Day_Most_Recent_Flight_ArrDelay'] = shifted_delays['ArrDelay']
bay_area_flights_w_new_features['Plane_Same_Day_Most_Recent_Flight_DepDelay'] = shifted_delays['DepDelay']
bay_area_flights_w_new_features['Plane_Same_Day_Most_Recent_Flight_TotalDelay'] = shifted_delays['TotalDelay']

bay_area_flights_w_new_features.fillna({'Plane_Same_Day_Most_Recent_Flight_ArrDelay': 0,
                                        'Plane_Same_Day_Most_Recent_Flight_DepDelay': 0,
                                        'Plane_Same_Day_Most_Recent_Flight_TotalDelay': 0}, inplace=True)

print(bay_area_flights_w_new_features.head())

      DayOfWeek       Date   DepTime   ArrTime  CRSArrTime UniqueCarrier  \
39906   Tuesday 2019-01-01  08:17:00  11:02:00         915            US   
39858   Tuesday 2019-01-01  19:18:00  22:24:00        2121            US   
74314   Tuesday 2019-01-01  19:25:00  21:04:00        2045            AA   
45835   Tuesday 2019-01-01  20:59:00  22:12:00        2150            WN   
29239   Tuesday 2019-01-01  16:42:00  23:46:00        2115            UA   

                      Airline  FlightNum TailNum  ActualElapsedTime  ...  \
39906         US Airways Inc.        202  N164AW                105  ...   
39858         US Airways Inc.        703  N171US                366  ...   
74314  American Airlines Inc.       1303  N201AA                219  ...   
45835  Southwest Airlines Co.       2285  N202WN                 73  ...   
29239   United Air Lines Inc.         74  N213UA                304  ...   

       NASDelay  SecurityDelay  LateAircraftDelay  DepHour Month  \
39906         0   

## Focusing on San Francisco Bay Area Departures

This section refines the data to concentrate on flights originating from airports within the San Francisco Bay Area.

**Considering Existing Filtering:**

- We've previously filtered the data to include flights either departing from or arriving at a Bay Area airport (`bay_area_flights`).

- While the previous filtering considered both arrival and departure airports, our project's focus might be on understanding delays specifically for flights departing from the Bay Area.

**Implementation:** Conditional Filtering
    
- Create a new DataFrame containing only flights where `'Origin'` is in `bay_area_airports` if focusing solely on Bay Area departures is desired.


In [ ]:
bay_area_airports = ['OAK', 'SFO', 'SJC']
bay_area_flights_w_new_features = bay_area_flights_w_new_features[bay_area_flights_w_new_features['Origin'].isin(bay_area_airports)]
print(bay_area_flights_w_new_features.shape)
print(bay_area_flights_w_new_features.head())

(23838, 33)
      DayOfWeek       Date   DepTime   ArrTime  CRSArrTime UniqueCarrier  \
39906   Tuesday 2019-01-01  08:17:00  11:02:00         915            US   
45835   Tuesday 2019-01-01  20:59:00  22:12:00        2150            WN   
29232   Tuesday 2019-01-01  15:20:00  18:30:00        1230            UA   
17167   Tuesday 2019-01-01  11:49:00  12:26:00        1137            OO   
17154   Tuesday 2019-01-01  13:45:00  14:53:00        1436            OO   

                      Airline  FlightNum TailNum  ActualElapsedTime  ...  \
39906         US Airways Inc.        202  N164AW                105  ...   
45835  Southwest Airlines Co.       2285  N202WN                 73  ...   
29232   United Air Lines Inc.         73  N214UA                310  ...   
17167   Skywest Airlines Inc.       5741  N218SW                 37  ...   
17154   Skywest Airlines Inc.       5478  N218SW                 68  ...   

       NASDelay  SecurityDelay  LateAircraftDelay  DepHour Month  \
39906 


This section explores incorporating new features related to cumulative delays.

**Goal:**

- Understanding the overall delay patterns at airports or for airlines on a given day.

1. **Cumulative Delay by Airport:**
   - We can calculate the total delay for all flights departing from a specific airport on a given day.
   - This involves grouping the data by `'Date'` and `'Origin'`, then summing the `'Delay'` column within each group.

2. **Cumulative Delay by Airline:**
   - Similarly, grouping by `'Date'` and `'Airline'` can provide the total delay for each airline on a particular day.
   - Summing the `'Delay'` column within these groups would yield the cumulative delay per airline per day.

**Implementation Using `groupby`:**
   - Group the data by the desired criteria (`'Date'` and `'Origin'` or `'Airline'`) and calculate the sum of delays within each group.



In [ ]:
bay_area_flights_w_new_features['Cumulative_ArrDelay_Before'] = bay_area_flights_w_new_features.groupby(['Date', 'Origin'])['ArrDelay'].transform(lambda x: x.cumsum() - x)
bay_area_flights_w_new_features['Cumulative_DepDelay_Before'] = bay_area_flights_w_new_features.groupby(['Date', 'Origin'])['DepDelay'].transform(lambda x: x.cumsum() - x)
bay_area_flights_w_new_features['Cumulative_ArrDep_Delay_Before'] = bay_area_flights_w_new_features['Cumulative_ArrDelay_Before'] + bay_area_flights_w_new_features['Cumulative_DepDelay_Before']

bay_area_flights_w_new_features['Cumulative_ArrDelay_Before_Same_Airline'] = bay_area_flights_w_new_features.groupby(['Date', 'Airline', 'Origin'])['ArrDelay'].transform(lambda x: x.cumsum() - x)
bay_area_flights_w_new_features['Cumulative_DepDelay_Before_Same_Airline'] = bay_area_flights_w_new_features.groupby(['Date', 'Airline', 'Origin'])['DepDelay'].transform(lambda x: x.cumsum() - x)
bay_area_flights_w_new_features['Cumulative_ArrDep_Delay_Before_Same_Airline'] = bay_area_flights_w_new_features['Cumulative_ArrDelay_Before_Same_Airline'] + bay_area_flights_w_new_features['Cumulative_DepDelay_Before_Same_Airline']

print(bay_area_flights_w_new_features[['Date', 'Origin', 'Airline', 'DepTime', 'ArrDelay', 'DepDelay', 'Cumulative_ArrDep_Delay_Before', 'Cumulative_ArrDep_Delay_Before_Same_Airline']].head())

            Date Origin                 Airline   DepTime  ArrDelay  DepDelay  \
39906 2019-01-01    SJC         US Airways Inc.  08:17:00       107       122   
45835 2019-01-01    SJC  Southwest Airlines Co.  20:59:00        22        19   
29232 2019-01-01    SFO   United Air Lines Inc.  15:20:00       360       373   
17167 2019-01-01    SFO   Skywest Airlines Inc.  11:49:00        49        54   
17154 2019-01-01    SFO   Skywest Airlines Inc.  13:45:00        17        30   

       Cumulative_ArrDep_Delay_Before  \
39906                               0   
45835                             229   
29232                               0   
17167                             733   
17154                             836   

       Cumulative_ArrDep_Delay_Before_Same_Airline  
39906                                            0  
45835                                            0  
29232                                            0  
17167                                            0  
1

This section introduces additional features to the `bay_area_flights_w_new_features` DataFrame, potentially aiding in understanding how a flight's position within the day's schedule might influence delays.

**Features Added:**

1. **Flights Before:**
   - `'Flights_Before'` represents the number of flights departing from the same airport (`'Origin'`) on the same day (`'Date'`) that occurred before the current flight based on departure time (`'DepTime'`).
   - This is calculated using `groupby` and `cumcount` to assign a sequential number within each origin-date group.

2. **Flights Before (Same Airline):**
   - `'Flights_Before_Same_Airline'` extends the concept of `'Flights_Before'`, but considers only flights from the same airline (`'Airline'`) departing from the same airport (`'Origin'`) before the current flight.
   - This is computed similarly using `groupby` and `cumcount` within groups defined by `'Date'`, `'Airline'`, and `'Origin'`.

3. **Average Arrival Delay Before:**
   - `'Avg_ArrDelay_Before'` calculates the average arrival delay (`'ArrDelay'`) of flights departing from the same airport (`'Origin'`) on the same day (`'Date'`) that occurred before the current flight.
   - This leverages the `transform` function with a lambda expression.
     - The expression calculates the cumulative sum of arrival delays minus the current flight's delay.
     - It then divides by the rolling count of flights (excluding the current flight) within the same origin-date group using `rolling` and `count`.

4. **Average Departure Delay Before:**
   - `'Avg_DepDelay_Before'` employs the same logic as `'Avg_ArrDelay_Before'` but for departure delays (`'DepDelay'`).

5. **Average Arrival Delay Before (Same Airline):**
   - `'Avg_ArrDelay_Before_Same_Airline'` is similar to `'Avg_ArrDelay_Before'` but considers only flights from the same airline (`'Airline'`) departing from the same airport (`'Origin'`) before the current flight.

6. **Average Departure Delay Before (Same Airline):**
   - `'Avg_DepDelay_Before_Same_Airline'` mirrors `'Avg_ArrDelay_Before_Same_Airline'` for departure delays (`'DepDelay'`).

**Benefits:**

- These features capture the context of a flight within the day's schedule at a specific airport (both overall and for the same airline).
- They might provide insights into potential delay propagation or dependencies between flights.



In [ ]:
bay_area_flights_w_new_features.sort_values(by=['Date', 'Origin', 'DepTime'], inplace=True)
bay_area_flights_w_new_features['Flights_Before'] = bay_area_flights_w_new_features.groupby(['Date', 'Origin']).cumcount()
bay_area_flights_w_new_features['Flights_Before_Same_Airline'] = bay_area_flights_w_new_features.groupby(['Date', 'Airline', 'Origin']).cumcount()
bay_area_flights_w_new_features['Avg_ArrDelay_Before'] = bay_area_flights_w_new_features.groupby(['Date', 'Origin'])['ArrDelay'].transform(lambda x: (x.cumsum() - x) / (x.rolling(window=len(x), min_periods=1).count() - 1))
bay_area_flights_w_new_features['Avg_DepDelay_Before'] = bay_area_flights_w_new_features.groupby(['Date', 'Origin'])['DepDelay'].transform(lambda x: (x.cumsum() - x) / (x.rolling(window=len(x), min_periods=1).count() - 1))
bay_area_flights_w_new_features['Avg_ArrDelay_Before_Same_Airline'] = bay_area_flights_w_new_features.groupby(['Date', 'Airline', 'Origin'])['ArrDelay'].transform(lambda x: (x.cumsum() - x) / (x.rolling(window=len(x), min_periods=1).count() - 1))
bay_area_flights_w_new_features['Avg_DepDelay_Before_Same_Airline'] = bay_area_flights_w_new_features.groupby(['Date', 'Airline', 'Origin'])['DepDelay'].transform(lambda x: (x.cumsum() - x) / (x.rolling(window=len(x), min_periods=1).count() - 1))

print(bay_area_flights_w_new_features[['Date', 'Origin', 'Airline', 'DepTime', 'ArrDelay', 'DepDelay', 'Flights_Before', 'Flights_Before_Same_Airline', 'Avg_ArrDelay_Before', 'Avg_DepDelay_Before', 'Avg_ArrDelay_Before_Same_Airline', 'Avg_DepDelay_Before_Same_Airline']].head())

            Date Origin                 Airline   DepTime  ArrDelay  DepDelay  \
29463 2019-01-01    OAK   United Air Lines Inc.  00:03:00        31        48   
34277 2019-01-01    OAK   United Air Lines Inc.  06:28:00        16        19   
45646 2019-01-01    OAK  Southwest Airlines Co.  08:22:00        18        22   
80690 2019-01-01    OAK    Alaska Airlines Inc.  08:54:00        50        59   
45664 2019-01-01    OAK  Southwest Airlines Co.  09:05:00        24        25   

       Flights_Before  Flights_Before_Same_Airline  Avg_ArrDelay_Before  \
29463               0                            0                  NaN   
34277               1                            1            31.000000   
45646               2                            0            23.500000   
80690               3                            0            21.666667   
45664               4                            1            28.750000   

       Avg_DepDelay_Before  Avg_ArrDelay_Before_Same_Airline  

This section addresses the potential `NaN` values introduced during feature creation in `bay_area_flights_w_new_features`.

**Reason for NaNs:**
- These NaNs arise from the division by zero when calculating average delays before the first flight on a given day.If there are no preceding flights, there will be a zero denominator during division.

**Imputation Strategy:**

- We'll replace these `NaN` values with `0` since the first flight of the day wouldn't have any delays from previous flights to influence its own delay.

**Implementation:**
   - A list named `columns_to_replace_nan` stores the column names containing potential `NaN` values.
   - We iterate through the columns in the list using a `for` loop.
   - Inside the loop, the `fillna(0)` method is applied to each column, replacing any `NaN` values with `0`.

**Goal:**

- Ensures all features have valid values for further analysis, and maintains the interpretation that the first flight of the day has no preceding flight delays.


In [ ]:
columns_to_replace_nan = ['Avg_ArrDelay_Before', 'Avg_DepDelay_Before',
                          'Avg_ArrDelay_Before_Same_Airline', 'Avg_DepDelay_Before_Same_Airline']
for column in columns_to_replace_nan:
    bay_area_flights_w_new_features[column] = bay_area_flights_w_new_features[column].fillna(0)

print(bay_area_flights_w_new_features[['Date', 'Origin', 'Airline', 'DepTime', 'ArrDelay', 'DepDelay',
                        'Flights_Before', 'Flights_Before_Same_Airline', 'Avg_ArrDelay_Before',
                        'Avg_DepDelay_Before', 'Avg_ArrDelay_Before_Same_Airline',
                        'Avg_DepDelay_Before_Same_Airline']].head())

            Date Origin                 Airline   DepTime  ArrDelay  DepDelay  \
29463 2019-01-01    OAK   United Air Lines Inc.  00:03:00        31        48   
34277 2019-01-01    OAK   United Air Lines Inc.  06:28:00        16        19   
45646 2019-01-01    OAK  Southwest Airlines Co.  08:22:00        18        22   
80690 2019-01-01    OAK    Alaska Airlines Inc.  08:54:00        50        59   
45664 2019-01-01    OAK  Southwest Airlines Co.  09:05:00        24        25   

       Flights_Before  Flights_Before_Same_Airline  Avg_ArrDelay_Before  \
29463               0                            0             0.000000   
34277               1                            1            31.000000   
45646               2                            0            23.500000   
80690               3                            0            21.666667   
45664               4                            1            28.750000   

       Avg_DepDelay_Before  Avg_ArrDelay_Before_Same_Airline  

Let's check what features we have so far:

In [ ]:
bay_area_flights_w_new_features.columns

Index(['DayOfWeek', 'Date', 'DepTime', 'ArrTime', 'CRSArrTime',
       'UniqueCarrier', 'Airline', 'FlightNum', 'TailNum', 'ActualElapsedTime',
       'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin',
       'Org_Airport', 'Dest', 'Dest_Airport', 'Distance', 'TaxiIn', 'TaxiOut',
       'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay', 'DepHour', 'Month',
       'Same_Day_Num_Flights_of_this_Plane', 'TotalDelay',
       'Plane_Same_Day_Most_Recent_Flight_ArrDelay',
       'Plane_Same_Day_Most_Recent_Flight_DepDelay',
       'Plane_Same_Day_Most_Recent_Flight_TotalDelay',
       'Cumulative_ArrDelay_Before', 'Cumulative_DepDelay_Before',
       'Cumulative_ArrDep_Delay_Before',
       'Cumulative_ArrDelay_Before_Same_Airline',
       'Cumulative_DepDelay_Before_Same_Airline',
       'Cumulative_ArrDep_Delay_Before_Same_Airline', 'Flights_Before',
       'Flights_Before_Same_Airline', 'Avg_ArrDelay_Before',
       'Avg_DepDelay_Before', 

Store it in the Drive folder for better retrieval in the future:

In [ ]:
bay_area_flights_w_new_features.to_csv('/content/drive/My Drive/ieor 142 final project/Flight Delay/Transformed Data/Bay_Area_Flights.csv', index=False)
bay_area_flights_w_new_features.head()

,DayOfWeek,Date,DepTime,ArrTime,CRSArrTime,UniqueCarrier,Airline,FlightNum,TailNum,ActualElapsedTime,...,Cumulative_ArrDep_Delay_Before,Cumulative_ArrDelay_Before_Same_Airline,Cumulative_DepDelay_Before_Same_Airline,Cumulative_ArrDep_Delay_Before_Same_Airline,Flights_Before,Flights_Before_Same_Airline,Avg_ArrDelay_Before,Avg_DepDelay_Before,Avg_ArrDelay_Before_Same_Airline,Avg_DepDelay_Before_Same_Airline
29463,Tuesday,2019-01-01,00:03:00,07:48:00,717,UA,United Air Lines Inc.,112,N437UA,285,...,1387,0,0,0,0,0,0.000000,0.000000,0.0,0.0
34277,Tuesday,2019-01-01,06:28:00,12:33:00,1217,UA,United Air Lines Inc.,652,N803UA,245,...,3872,31,48,79,1,1,31.000000,48.000000,31.0,48.0
45646,Tuesday,2019-01-01,08:22:00,09:33:00,915,WN,Southwest Airlines Co.,3245,N249WN,71,...,475,229,246,475,2,0,23.500000,33.500000,0.0,0.0
80690,Tuesday,2019-01-01,08:54:00,10:23:00,933,AS,Alaska Airlines Inc.,407,N969AS,89,...,4115,56,56,112,3,0,21.666667,29.666667,0.0,0.0
45664,Tuesday,2019-01-01,09:05:00,10:24:00,1000,WN,Southwest Airlines Co.,3570,N736SA,79,...,3468,1198,1227,2425,4,1,28.750000,37.000000,18.0,22.0


#### Bay Area Airports Weather Data

## Incorporating Weather Data

This section explores integrating weather data from the National Centers for Environmental Information (NCEI)  (https://www.ncdc.noaa.gov/cdo-web/search) into our analysis of flight delays.

**Cherrypick Weather Data:**

- We selected the 3 ZIP codes (94128, 94621, 95110) that the three bay area airports are in for the entire Year 2019.
- Upon closer inspection, these ZIP codes are confined to the immediate vicinity of the Bay Area airports.

**Justification:**

- Weather conditions can significantly impact flight operations and delays. We can identify weather-related factors influencing flight delays, specifically the geographical regions surrounding the Bay Area airports.

In [ ]:
print(bay_weather.shape)
bay_weather.head()

(1095, 20)


,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT05,WT08
0,USW00023293,"SAN JOSE, CA US",2019-01-01,5.82,NaN,0.0,NaN,NaN,NaN,55.0,36.0,50,60.0,17.0,23.0,NaN,NaN,NaN,NaN,NaN
1,USW00023293,"SAN JOSE, CA US",2019-01-02,4.03,NaN,0.0,NaN,NaN,NaN,57.0,33.0,330,330.0,8.9,10.1,NaN,NaN,NaN,NaN,NaN
2,USW00023293,"SAN JOSE, CA US",2019-01-03,2.68,NaN,0.0,NaN,NaN,NaN,58.0,34.0,320,340.0,8.9,11.0,NaN,NaN,NaN,NaN,NaN
3,USW00023293,"SAN JOSE, CA US",2019-01-04,3.36,NaN,0.0,NaN,NaN,NaN,62.0,35.0,200,200.0,14.1,19.0,NaN,NaN,NaN,NaN,NaN
4,USW00023293,"SAN JOSE, CA US",2019-01-05,13.20,NaN,0.1,NaN,NaN,NaN,56.0,46.0,130,150.0,30.0,40.0,1.0,NaN,NaN,NaN,1.0


Let's inspect the distribution of missing values across features:

In [ ]:
nan_counts = bay_weather.isna().sum()
print(nan_counts)

STATION       0
NAME          0
DATE          0
AWND          1
PGTM        997
PRCP          0
SNOW        788
SNWD        736
TAVG        730
TMAX          3
TMIN          1
WDF2          0
WDF5          7
WSF2          0
WSF5          7
WT01        763
WT02       1077
WT03       1078
WT05       1095
WT08        817
dtype: int64


## Refining Weather Data Preprocessing

This section addresses the cleaning and preparation of weather data (`bay_weather`) for merging with the flight data.

**Steps:**

1. **Identifying Columns for Removal:**
   - A threshold is defined based on the total number of rows in the data (`len(bay_weather)`) divided by 5.
   - Columns with missing values (`isna().sum()`) exceeding this threshold are identified and stored in a list `cols_to_remove`.
   - These columns likely contain too many missing values to be reliable, so they are removed using `drop`.

2. **Column Renaming:**
   - A dictionary `renaming_dict` maps abbreviations used in the weather data to more descriptive names.
   - The `rename` function is applied to `bay_weather_cleaned` to replace the short codes with their corresponding human-readable names (e.g., `'PRCP'` to `'Precipitation'`).

3. **Extracting City and Airport Code:**
   - A function `extract_city` is defined to process the `'NAME'` column.
   - It checks if the city name (`name`) contains keywords like "OAKLAND", "SAN JOSE", or "SAN FRANCISCO".
   - Based on the match, it returns a tuple with the city name and corresponding airport code (`'OAK'`, `'SJC'`, or `'SFO'`).
   - If no match is found, it returns `np.nan` for both city and code.
   - Two new columns, `'Airport City'` and `'Airport Code'`, are created using vectorized operations (`apply` and `zip`) to extract city names and airport codes from the `'NAME'` column using the `extract_city` function.

4. **Dropping Unnecessary Columns:**
   - Columns `'STATION'`, `'NAME'`, and `'Airport City'`, redundant after code extraction, are dropped using `drop`.

**Improvements:**

- Removing columns with excessive missing values reduces the impact of potentially unreliable data.
- Descriptive column names enhance readability and understanding of the weather data.
- Extracted city names and airport codes facilitate merging with the flight data based on airport locations.

In [ ]:
threshold = len(bay_weather) // 5
cols_to_remove = [col for col in bay_weather.columns if bay_weather[col].isna().sum() > threshold]
bay_weather_cleaned = bay_weather.drop(cols_to_remove, axis=1)
renaming_dict = {
    'PRCP': 'Precipitation',
    'SNWD': 'Snow_Depth',
    'SNOW': 'Snowfall',
    'TAVG': 'Average_Temperature',
    'TMAX': 'Maximum_Temperature',
    'TMIN': 'Minimum_Temperature',
    'AWND': 'Average_Wind_Speed',
    'WDF2': 'Direction_Fastest_2min_Wind',
    'WDF5': 'Direction_Fastest_5sec_Wind',
    'WSF2': 'Fastest_2min_Wind_Speed',
    'WSF5': 'Fastest_5sec_Wind_Speed',
    'PGTM': 'Peak_Gust_Time',
}

bay_weather_cleaned = bay_weather_cleaned.rename(columns=renaming_dict)

def extract_city(name):
    if 'OAKLAND' in name:
        return 'Oakland', 'OAK'
    elif 'SAN JOSE' in name:
        return 'San Jose', 'SJC'
    elif 'SAN FRANCISCO' in name:
        return 'San Francisco', 'SFO'
    return np.nan, np.nan

bay_weather_cleaned['Airport City'], bay_weather_cleaned['Airport Code'] = zip(*bay_weather_cleaned['NAME'].apply(extract_city))
bay_weather_cleaned.drop(columns=['STATION', 'NAME', 'Airport City'], inplace=True)
bay_weather_cleaned.head()

,DATE,Average_Wind_Speed,Precipitation,Maximum_Temperature,Minimum_Temperature,Direction_Fastest_2min_Wind,Direction_Fastest_5sec_Wind,Fastest_2min_Wind_Speed,Fastest_5sec_Wind_Speed,Airport Code
0,2019-01-01,5.82,0.0,55.0,36.0,50,60.0,17.0,23.0,SJC
1,2019-01-02,4.03,0.0,57.0,33.0,330,330.0,8.9,10.1,SJC
2,2019-01-03,2.68,0.0,58.0,34.0,320,340.0,8.9,11.0,SJC
3,2019-01-04,3.36,0.0,62.0,35.0,200,200.0,14.1,19.0,SJC
4,2019-01-05,13.20,0.1,56.0,46.0,130,150.0,30.0,40.0,SJC


In [ ]:
bay_weather_cleaned['Airport Code'].unique()

array(['SJC', 'OAK', 'SFO'], dtype=object)

In [ ]:
nan_counts = bay_weather_cleaned.isna().sum()
print(nan_counts)

DATE                           0
Average_Wind_Speed             1
Precipitation                  0
Maximum_Temperature            3
Minimum_Temperature            1
Direction_Fastest_2min_Wind    0
Direction_Fastest_5sec_Wind    7
Fastest_2min_Wind_Speed        0
Fastest_5sec_Wind_Speed        7
Airport Code                   0
dtype: int64


Here, we still have missing values for some features. But let's not worry about it now since we can simply remove the rows that contain them in the merged dataset because merging can result in other missing values as well and it will be more efficient to handle with them together:

#### Merging the datasets

Let's merge our transformed flight delay data and the airport weather data:

In [ ]:
bay_weather_cleaned.rename(columns={'DATE': 'Date', 'Airport Code': 'Origin'}, inplace=True)

bay_area_flights_w_new_features['Date'] = pd.to_datetime(bay_area_flights_w_new_features['Date'])
bay_weather_cleaned['Date'] = pd.to_datetime(bay_weather_cleaned['Date'])
bay_area_flights_w_new_features['Origin'] = bay_area_flights_w_new_features['Origin'].astype(str)
bay_weather_cleaned['Origin'] = bay_weather_cleaned['Origin'].astype(str)

merged_data = pd.merge(bay_area_flights_w_new_features, bay_weather_cleaned, on=['Date', 'Origin'], how='left')
print(merged_data.shape)
print(merged_data.head())

(23838, 53)
  DayOfWeek       Date   DepTime   ArrTime  CRSArrTime UniqueCarrier  \
0   Tuesday 2019-01-01  00:03:00  07:48:00         717            UA   
1   Tuesday 2019-01-01  06:28:00  12:33:00        1217            UA   
2   Tuesday 2019-01-01  08:22:00  09:33:00         915            WN   
3   Tuesday 2019-01-01  08:54:00  10:23:00         933            AS   
4   Tuesday 2019-01-01  09:05:00  10:24:00        1000            WN   

                  Airline  FlightNum TailNum  ActualElapsedTime  ...  \
0   United Air Lines Inc.        112  N437UA                285  ...   
1   United Air Lines Inc.        652  N803UA                245  ...   
2  Southwest Airlines Co.       3245  N249WN                 71  ...   
3    Alaska Airlines Inc.        407  N969AS                 89  ...   
4  Southwest Airlines Co.       3570  N736SA                 79  ...   

   Avg_ArrDelay_Before_Same_Airline  Avg_DepDelay_Before_Same_Airline  \
0                               0.0              

In [ ]:
merged_data.columns

Index(['DayOfWeek', 'Date', 'DepTime', 'ArrTime', 'CRSArrTime',
       'UniqueCarrier', 'Airline', 'FlightNum', 'TailNum', 'ActualElapsedTime',
       'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin',
       'Org_Airport', 'Dest', 'Dest_Airport', 'Distance', 'TaxiIn', 'TaxiOut',
       'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay', 'DepHour', 'Month',
       'Same_Day_Num_Flights_of_this_Plane', 'TotalDelay',
       'Plane_Same_Day_Most_Recent_Flight_ArrDelay',
       'Plane_Same_Day_Most_Recent_Flight_DepDelay',
       'Plane_Same_Day_Most_Recent_Flight_TotalDelay',
       'Cumulative_ArrDelay_Before', 'Cumulative_DepDelay_Before',
       'Cumulative_ArrDep_Delay_Before',
       'Cumulative_ArrDelay_Before_Same_Airline',
       'Cumulative_DepDelay_Before_Same_Airline',
       'Cumulative_ArrDep_Delay_Before_Same_Airline', 'Flights_Before',
       'Flights_Before_Same_Airline', 'Avg_ArrDelay_Before',
       'Avg_DepDelay_Before', 

## Handling Missing Weather Data

This section addresses the presence of missing values in the weather data (`bay_weather_cleaned`) after merging it with the flight data (`bay_area_flights_w_new_features`) to create `merged_data`.

**Justification for Removal:**

- We decided to remove rows containing missing values in specific weather columns (`columns_to_check`).
- The rationale behind this approach is the assumption that eliminating these rows won't significantly impact the data distribution or introduce bias.

**Implementation:**

1. **Columns to Check:**
   - A list named `columns_to_check` identifies the weather-related columns where missing values will be addressed.

2. **Dropping Rows with Missing Values:**
   - The `dropna` function is applied to `merged_data`, specifying the `subset` parameter as `columns_to_check`.
   - This removes rows where any of the listed weather columns contain missing values.


In [ ]:
columns_to_check = [
    'Average_Wind_Speed', 'Precipitation', 'Maximum_Temperature',
    'Minimum_Temperature', 'Direction_Fastest_2min_Wind',
    'Direction_Fastest_5sec_Wind', 'Fastest_2min_Wind_Speed',
    'Fastest_5sec_Wind_Speed'
]
merged_data_cleaned = merged_data.dropna(subset=columns_to_check)

print(merged_data_cleaned.shape)
print(merged_data_cleaned.head())

(23506, 53)
  DayOfWeek       Date   DepTime   ArrTime  CRSArrTime UniqueCarrier  \
0   Tuesday 2019-01-01  00:03:00  07:48:00         717            UA   
1   Tuesday 2019-01-01  06:28:00  12:33:00        1217            UA   
2   Tuesday 2019-01-01  08:22:00  09:33:00         915            WN   
3   Tuesday 2019-01-01  08:54:00  10:23:00         933            AS   
4   Tuesday 2019-01-01  09:05:00  10:24:00        1000            WN   

                  Airline  FlightNum TailNum  ActualElapsedTime  ...  \
0   United Air Lines Inc.        112  N437UA                285  ...   
1   United Air Lines Inc.        652  N803UA                245  ...   
2  Southwest Airlines Co.       3245  N249WN                 71  ...   
3    Alaska Airlines Inc.        407  N969AS                 89  ...   
4  Southwest Airlines Co.       3570  N736SA                 79  ...   

   Avg_ArrDelay_Before_Same_Airline  Avg_DepDelay_Before_Same_Airline  \
0                               0.0              

## Saving Merged Data for Later Use
- The EDA and Data pre-processing part is completed for now.
- Now store the preprocessed and merged data (merged_data_cleaned) into Google Drive for easy access in our modeling notebook.
- ***The modeling will be performed in a separate notebook in the folder for clearer understanding***.

In [ ]:
merged_data_cleaned.to_csv('/content/drive/My Drive/ieor 142 final project/Flight Delay/Transformed Data/Bay_Area_Flights_feature_merged.csv', index=False)